In [1]:
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer

In [8]:
review_df = pd.read_json('../Review100k.json')

In [9]:
review_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
99995,pAEbIxvr6ebx2bHc1XvguA,SMH5CeiLvKx61lKwtLZ_PA,lV0k3BnslFRkuWD_kbKd0Q,4,0,0,0,Came here for lunch with a group. They were bu...,2018-05-30 22:28:56
99996,xH1AoE-4nf2ECGQJRjO4_g,2clTdtp-BjphxLjN83CpUA,G0xz3kyRhRi6oZl7KfR0pA,1,1,0,0,The equipment is so old and so felty! I just u...,2015-04-05 23:31:52
99997,GatIbXTz-WDru5emONUSIg,MRrN6DH3QGCFcDv5RENYVg,C4lZdhasjZVQyDlOiXY1sA,4,0,0,0,This is one of my favorite Mexican restaurants...,2016-06-04 00:59:15
99998,6NfkodAdhvI89xONXuBC3A,rnNQzeKJbvqVCsYsL10mkQ,dChRGpit9fM_kZK5pafNyA,2,0,0,0,Came here for brunch - had an omlette ($19 + t...,2018-06-11 12:45:08


In [12]:
review_df.dtypes

review_id              object
user_id                object
business_id            object
stars                   int64
useful                  int64
funny                   int64
cool                    int64
text                   object
date           datetime64[ns]
dtype: object

In [10]:
business_df = pd.read_json('../Business100K.json')

In [11]:
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,XwEzff2lxcn2MoWdRaMqUg,The Groom Room,210 Louise Ave,Nashville,TN,37203,36.151926,-86.804949,4.5,23,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Waxing, Spray Tanning, Permanent Makeup, Eyela...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-19:0', ..."
99996,BjRz_MdoqUn6FYDsjAqn6Q,61Five Health & Wellness,1913 Church St,Nashville,TN,37203,36.153587,-86.799671,5.0,78,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","IV Hydration, Beauty & Spas, Medical Centers, ...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."
99997,2pWRW7ASh_Bw5q5V64Luuw,Ridge Carlton Apartments,7373 Ridge Ave,Philadelphia,PA,19128,40.047768,-75.234005,2.5,9,1,{'BusinessAcceptsCreditCards': 'True'},"Real Estate, Home Services, Apartments, Real E...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
99998,ccwhUs2fCLWTQLMa1L_xNQ,China Buffet,57 Fenton Plz,Fenton,MO,63026,38.511352,-90.442368,2.0,10,0,"{'RestaurantsAttire': 'u'casual'', 'GoodForKid...","Chinese, Restaurants",None


In [13]:
business_df.dtypes

business_id      object
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
stars           float64
review_count      int64
is_open           int64
attributes       object
categories       object
hours            object
dtype: object

In [25]:
merged_df = review_df.merge(business_df, how = 'left', on = ['business_id'])

In [26]:
merged_df

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,Turning Point of North Wales,...,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,Body Cycle Spinning Studio,...,PA,19119,39.952103,-75.172753,5.0,144,0,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","Active Life, Cycling Classes, Trainers, Gyms, ...","{'Monday': '6:30-20:30', 'Tuesday': '6:30-20:3..."
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,Kettle Restaurant,...,AZ,85713,32.207233,-110.980864,3.5,47,1,"{'RestaurantsReservations': 'True', 'BusinessP...","Restaurants, Breakfast & Brunch",None
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,Zaika,...,PA,19114,40.079848,-75.025080,4.0,181,1,"{'Caters': 'True', 'Ambience': '{'romantic': F...","Halal, Pakistani, Restaurants, Indian","{'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21..."
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,Melt,...,LA,70119,29.962102,-90.087958,4.0,32,0,"{'BusinessParking': '{'garage': False, 'street...","Sandwiches, Beer, Wine & Spirits, Bars, Food, ...","{'Monday': '0:0-0:0', 'Friday': '11:0-17:0', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,pAEbIxvr6ebx2bHc1XvguA,SMH5CeiLvKx61lKwtLZ_PA,lV0k3BnslFRkuWD_kbKd0Q,4,0,0,0,Came here for lunch with a group. They were bu...,2018-05-30 22:28:56,Metro Diner,...,IN,46260,39.913046,-86.200355,4.0,175,0,"{'RestaurantsTableService': 'True', 'OutdoorSe...","American (Traditional), Breakfast & Brunch, Re...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-20:0', '..."
99996,xH1AoE-4nf2ECGQJRjO4_g,2clTdtp-BjphxLjN83CpUA,G0xz3kyRhRi6oZl7KfR0pA,1,1,0,0,The equipment is so old and so felty! I just u...,2015-04-05 23:31:52,Esporta Fitness,...,PA,19401,40.147359,-75.318160,2.5,55,1,"{'GoodForKids': 'False', 'ByAppointmentOnly': ...","Yoga, Gyms, Trainers, Fitness & Instruction, A...","{'Monday': '0:0-0:0', 'Tuesday': '5:0-22:0', '..."
99997,GatIbXTz-WDru5emONUSIg,MRrN6DH3QGCFcDv5RENYVg,C4lZdhasjZVQyDlOiXY1sA,4,0,0,0,This is one of my favorite Mexican restaurants...,2016-06-04 00:59:15,Buena Vista Mexican Restaurant,...,PA,19355,40.042104,-75.541083,3.5,107,1,"{'NoiseLevel': 'u'quiet'', 'RestaurantsTakeOut...","Mexican, Restaurants","{'Monday': '11:30-21:0', 'Tuesday': '11:30-21:..."
99998,6NfkodAdhvI89xONXuBC3A,rnNQzeKJbvqVCsYsL10mkQ,dChRGpit9fM_kZK5pafNyA,2,0,0,0,Came here for brunch - had an omlette ($19 + t...,2018-06-11 12:45:08,The Love,...,PA,19103,39.950656,-75.170899,4.0,618,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Restaurants, American (New), Breakfast & Brunc...","{'Monday': '0:0-0:0', 'Tuesday': '17:0-21:0', ..."


In [27]:
merged_df.dtypes

review_id               object
user_id                 object
business_id             object
stars_x                  int64
useful                   int64
funny                    int64
cool                     int64
text                    object
date            datetime64[ns]
name                    object
address                 object
city                    object
state                   object
postal_code             object
latitude               float64
longitude              float64
stars_y                float64
review_count             int64
is_open                  int64
attributes              object
categories              object
hours                   object
dtype: object

In [28]:
merged_df.isna().sum()

review_id          0
user_id            0
business_id        0
stars_x            0
useful             0
funny              0
cool               0
text               0
date               0
name               0
address            0
city               0
state              0
postal_code        0
latitude           0
longitude          0
stars_y            0
review_count       0
is_open            0
attributes      1631
categories         4
hours           5824
dtype: int64

In [29]:
S_B_Data =( merged_df[merged_df['postal_code'] == '93101'])

In [30]:
S_B_Data.dtypes
S_B_Data.isna().sum()

review_id         0
user_id           0
business_id       0
stars_x           0
useful            0
funny             0
cool              0
text              0
date              0
name              0
address           0
city              0
state             0
postal_code       0
latitude          0
longitude         0
stars_y           0
review_count      0
is_open           0
attributes       13
categories        0
hours           109
dtype: int64

In [31]:
S_B_Data.shape

(2346, 22)

In [43]:
S_B_Data.drop(['attributes', 'categories', 'user_id', 'useful', 'funny', 'cool', 'date'], axis=1)

,review_id,business_id,stars_x,text,name,address,city,state,postal_code,latitude,longitude,stars_y,review_count,is_open,hours
9,pUycOfUwM8vqX7KjRRhUEA,gebiRewfieSdtt17PTW6Zg,3,Had a party of 6 here for hibachi. Our waitres...,Hibachi Steak House & Sushi Bar,502 State St,Santa Barbara,CA,93101,34.416984,-119.695556,3.5,488,1,{'Monday': '0:0-0:0'}
35,L0jv8c2FbpWSlfNC6bbUEA,IDtLPgUrqorrpqSLdfMhZQ,5,What a great addition to the Funk Zone! Grab ...,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,34.414445,-119.690672,4.0,389,1,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '..."
56,2u5Skti5mZam_-XTKPelvA,qO9dNNIvNbCBd8ZgjxMxgQ,5,"Farmhouse, rustic, chic.Helpful staff with gre...",HOME Santa Barbara,14 Parker Way,Santa Barbara,CA,93101,34.414912,-119.694171,4.5,13,1,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
61,4zopEEPqfwm-c_FNpeHZYw,SZU9c8V2GuREDN5KgyHFJw,5,We were a bit weary about trying the Shellfish...,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ..."
70,HuWEnZr7-0HveaqXxyywgg,YbnJYHNp_fHbI-hcFg48vQ,5,The kayaking tour at the Santa Cruz Island was...,Santa Barbara Adventure Company,32 E Haley St,Santa Barbara,CA,93101,34.417681,-119.694349,5.0,195,1,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99796,8BGKq8dzsxlIClrm_lEQxg,SZU9c8V2GuREDN5KgyHFJw,3,local king crab for 18.95/lb is pretty tasty\n...,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ..."
99800,avAsbVWsLmXszusNkCW2Rg,uO39--k_hrCFgZh-Bl8m8A,5,"Great food. Full of locals. Clean, fast and fr...",Taqueria Cuernavaca,201 W Carrillo St,Santa Barbara,CA,93101,34.418932,-119.704239,4.5,409,1,"{'Monday': '10:0-20:30', 'Wednesday': '10:0-20..."
99867,28wmngyQguCcwiGv8NnLbQ,34Eqv8jXgxg_EEwcsNgeeg,2,Pretty disappointing as far as the SoCal acai ...,Backyard Bowls,331 Motor Way,Santa Barbara,CA,93101,34.415114,-119.694497,4.0,659,1,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
99886,Dw_ERN73HV97g1IpUsYkxw,B91yBSPvAz5O_t9JFafN1Q,5,My family has been coming here for generations...,O'Malleys Bar,523 State St,Santa Barbara,CA,93101,34.417389,-119.696584,2.5,172,1,"{'Monday': '12:0-1:30', 'Tuesday': '11:0-1:30'..."


In [44]:
S_B_Data['name'].value_counts()

Santa Barbara Shellfish Company    351
Dawn Patrol                        141
Backyard Bowls                     105
Hibachi Steak House & Sushi Bar     92
Taqueria Cuernavaca                 87
                                  ... 
Dalee Car Bath                       1
Howard Gross, MD                     1
Will Nelson Fitness                  1
Carrillo Union 76                    1
Rook Thomas, DC                      1
Name: name, Length: 144, dtype: int64

In [45]:
print(S_B_Data['name'])

9        Hibachi Steak House & Sushi Bar
35                  Helena Avenue Bakery
56                    HOME Santa Barbara
61       Santa Barbara Shellfish Company
70       Santa Barbara Adventure Company
                      ...               
99796    Santa Barbara Shellfish Company
99800                Taqueria Cuernavaca
99867                     Backyard Bowls
99886                      O'Malleys Bar
99937                    Tonic Nightclub
Name: name, Length: 2346, dtype: object


In [46]:
print(S_B_Data['name'].unique())

['Hibachi Steak House & Sushi Bar' 'Helena Avenue Bakery'
 'HOME Santa Barbara' 'Santa Barbara Shellfish Company'
 'Santa Barbara Adventure Company' 'Santa Barbara County Courthouse'
 'Topa Topa Brewing Company' 'Paradise Cafe' 'Backyard Bowls'
 'Dune Coffee Roasters - Anacapa' 'Deep Sea Tasting Room' "O'Malleys Bar"
 'Casa Del Mar Inn' 'University Club of Santa Barbara'
 'Amador Matchmaking' 'Dawn Patrol' 'Lama Dog Tap Room' 'Tuttini'
 'Zen Yai Thai Cuisine' 'Artistic Nails & Spa' 'Starbucks'
 "Maggie's at State and A" 'Breakfast Culture Club' 'Carrillo Union 76'
 'H&M' 'Urban Optics Optometric Center'
 'El Presidio de Santa Bárbara State Historic Park' 'Modern Laundry'
 "Wahoo's Fish Tacos" 'Taqueria Cuernavaca' 'Sushi Tyme'
 'Mail Box Express II' 'Tonic Nightclub' 'Cafe Buenos Aires'
 'Santa Barbara Jet Boats' 'Mary Tingaud, LAc' "Yoichi's" '19 Blue Salon'
 'Good Cup' 'The Eagle Inn' 'Kelsey Crews Photo'
 'Castillo Inn at the Beach' 'Santa Barbara Athletic Club'
 'So Good Jewelry' '

In [40]:
print(S_B_Data.name.unique().tolist())

['Hibachi Steak House & Sushi Bar', 'Helena Avenue Bakery', 'HOME Santa Barbara', 'Santa Barbara Shellfish Company', 'Santa Barbara Adventure Company', 'Santa Barbara County Courthouse', 'Topa Topa Brewing Company', 'Paradise Cafe', 'Backyard Bowls', 'Dune Coffee Roasters - Anacapa', 'Deep Sea Tasting Room', "O'Malleys Bar", 'Casa Del Mar Inn', 'University Club of Santa Barbara', 'Amador Matchmaking', 'Dawn Patrol', 'Lama Dog Tap Room', 'Tuttini', 'Zen Yai Thai Cuisine', 'Artistic Nails & Spa', 'Starbucks', "Maggie's at State and A", 'Breakfast Culture Club', 'Carrillo Union 76', 'H&M', 'Urban Optics Optometric Center', 'El Presidio de Santa Bárbara State Historic Park', 'Modern Laundry', "Wahoo's Fish Tacos", 'Taqueria Cuernavaca', 'Sushi Tyme', 'Mail Box Express II', 'Tonic Nightclub', 'Cafe Buenos Aires', 'Santa Barbara Jet Boats', 'Mary Tingaud, LAc', "Yoichi's", '19 Blue Salon', 'Good Cup', 'The Eagle Inn', 'Kelsey Crews Photo', 'Castillo Inn at the Beach', 'Santa Barbara Athletic

In [41]:
count = S_B_Data.name.unique().size

In [42]:
print(count)

144


In [51]:
S_B_Data.set_index()
S_B_Data

TypeError: set_index() missing 1 required positional argument: 'keys'

In [52]:
S_B_Data.to_csv('../Output_Folder/3_Table.csv', index=False)